# LangChain自查询(Self Query)功能
## 自查询(Self Query)简介
    自查询是LangChain提供的一种强大功能，它允许用户使用自然语言查询向量数据库，并自动将查询解析为：
        - 语义查询部分（用于向量搜索）
        - 结构化查询部分（用于过滤）
    
    这种方式结合了向量搜索的语义能力和传统数据库的过滤能力，使查询更加精准。
### 工作原理
        - 将自然语言查询拆分为搜索部分和过滤部分
        - 使用向量相似度搜索相关文档
        - 应用过滤条件进一步筛选结果
        - 返回最终匹配的文档
    
### 适用场景
        - 需要同时基于语义和元数据进行搜索的应用
        - 用户可能使用自然语言表达复杂查询条件的情况
        - RAG系统中需要精确过滤检索结果的场景

### 代码示例
注意：自查询检索器需要安装lark软件包，且不支持FAISS数据库，需要使用Chroma数据库

安装命令 pip install lark

In [2]:
# 导入必要的库
import os
import lark
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

## 1. 创建一个简单的文档列表
documents = [
    Document(page_content="一群蚂蚁在搬食物", metadata={"source": "文档1", "category": "自然", "author": "张三"}),
    Document(page_content="故事发生1920年，马邦德花钱买了一个老婆。", metadata={"source": "文档2", "category": "剧情", "author": "李四"}),
    Document(page_content="寻梦环游记，讲述的是一个关于梦想的故事。", metadata={"source": "文档3", "category": "动画", "author": "王五"}),
]

print(f"文档列表: {documents}")

## 2. 创建向量存储
embeddings = DashScopeEmbeddings(
    model="text-embedding-v2",
    dashscope_api_key=os.getenv("API_KEY"),
)
vector_store = Chroma.from_documents(documents, embedding=embeddings,persist_directory="ChromaDB/self_query")

print("向量数据库已创建并保存到本地")

## 3. 配置自查询检索器

llm = ChatOpenAI(
    model="qwen-turbo",
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("API_BASEURL"),
)

# 定义文档元数据属性
metadate_field_info = [
    AttributeInfo(
        name="source",
        description="文档来源",
        type="string",
        required=True,
    ),
    AttributeInfo(
        name="category",
        description="文档类型",
        type="string",
        required=True,
    ),
    AttributeInfo(
        name="author",
        description="文档作者",
        type="string",
        required=True,
    ),
]

# 文档内容描述
document_content_description = "文档基础信息"

# 创建自查询检索器
retriever = SelfQueryRetriever.from_llm(
    llm,
    vector_store,
    document_content_description,
    metadate_field_info,
    verbose=True,
    enable_limit=True,
    search_kwargs={"k": 2},
)

## 4. 测试自查询检索器
query = "请检索关于梦想的文档"

results = retriever.invoke(query)

print(f"检索结果: {results}")



文档列表: [Document(metadata={'source': '文档1', 'category': '自然', 'author': '张三'}, page_content='一群蚂蚁在搬食物'), Document(metadata={'source': '文档2', 'category': '剧情', 'author': '李四'}, page_content='故事发生1920年，马邦德花钱买了一个老婆。'), Document(metadata={'source': '文档3', 'category': '动画', 'author': '王五'}, page_content='寻梦环游记，讲述的是一个关于梦想的故事。')]
向量数据库已创建并保存到本地
检索结果: [Document(metadata={'author': '王五', 'category': '动画', 'source': '文档3'}, page_content='寻梦环游记，讲述的是一个关于梦想的故事。'), Document(metadata={'author': '王五', 'category': '动画', 'source': '文档3'}, page_content='寻梦环游记，讲述的是一个关于梦想的故事。')]
